In [ ]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
train_dir = '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images'
test_dir = '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_test_images'

In [ ]:
Name=[]
for file in os.listdir(train_dir):
    Name+=[file]
print(Name)
print(len(Name))

In [ ]:
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
trainset=[]
trainname=[]
for name in tqdm(Name):
    path=os.path.join(train_dir,name)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(150,150))
        image=img_to_array(image)
        image=image/255.0
        trainset+=[image]
        trainname+=[name]

In [ ]:
testset=[]
testname=[]
for name in tqdm(Name):
    path=os.path.join(test_dir,name)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(150,150))
        image=img_to_array(image)
        image=image/255.0
        testset+=[image]
        testname+=[name]

In [ ]:
n=len(trainset)
N=list(range(n))
random.seed(2021)
random.shuffle(N)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(15,15))
for i in range(9):
    r=i//3
    c=i%3
    img1=trainset[N[i]]
    name=trainname[N[i]]
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title(name)
    ax=axs[r][c].imshow(img1)
plt.show()

In [ ]:
trainX=np.array(trainset)
testX=np.array(testset)

In [ ]:
trainY0=pd.Series(trainname).map(normal_mapping)
testY0=pd.Series(testname).map(normal_mapping)

In [ ]:
trainY1=to_categorical(trainY0)
trainY=np.array(trainY1)

In [ ]:
trainx,testx,trainy,testy=train_test_split(trainX,trainY,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.5,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(150,150,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(len(Name), activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=50)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
pred2=model.predict(testX)

In [ ]:
PRED=[]
for item in pred2:
    value2=np.argmax(item)
    PRED+=[value2]
print(PRED[0:10])

ANS=list(testY0)
print(ANS[0:10])

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)